In [1]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from datetime import datetime, timedelta
from functools import reduce
import holidays
from sklearn.model_selection import train_test_split
from scipy.stats import kurtosis, skew
from load_data import load_data, split_data
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Data Prep


In [32]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from datetime import datetime, timedelta
from functools import reduce
import numpy as np
import pandas as pd
import holidays
from scipy.stats import kurtosis, skew


#load data
df_ntc = pd.read_csv("../data/NTC.csv", parse_dates=['datetime'])
df_ntc.set_index('datetime', inplace=True)
df_ntc.index = df_ntc.index.tz_localize('Europe/Brussels', ambiguous='infer')
df_ntc.index = df_ntc.index.tz_convert('UTC')

df_act_loss = pd.read_csv("../data/Avtice-losses.csv", skiprows=1, parse_dates=['Zeitstempel'], index_col='Zeitstempel')
df_act_loss = df_act_loss.rename(columns={"kWh": "mwh", "Zeitstempel": "datetime"})
df_act_loss.index.name = 'datetime'
df_act_loss.index = df_act_loss.index.tz_localize('Europe/Brussels', ambiguous='infer')
df_act_loss.index = df_act_loss.index.tz_convert('UTC')

df_weather_forecast = pd.read_csv("../data/Forecast-renewable-generation.csv", parse_dates=['datetime'])
df_weather_forecast.set_index('datetime', inplace=True)
df_weather_forecast.index = df_weather_forecast.index.tz_localize('Europe/Brussels', ambiguous='infer')
df_weather_forecast.index = df_weather_forecast.index.tz_convert('UTC')

df_weather_temp = pd.read_csv("../data/Forecast-temperature_new.csv", parse_dates=['datetime'])
df_weather_temp.set_index('datetime', inplace=True)
ambiguous_flags = [False] * len(df_weather_temp)  # Set all to False initially
ambiguous_idx = df_weather_temp.index.get_loc(pd.Timestamp("2019-10-27 02:00:00"))
ambiguous_flags[ambiguous_idx] = True  # Adjust as needed; True means it is DST
df_weather_temp.index = df_weather_temp.index.tz_localize('Europe/Brussels', ambiguous=ambiguous_flags)
df_weather_temp.index = df_weather_temp.index.tz_convert('UTC')
df_weather_temp = df_weather_temp.loc[~df_weather_temp.index.duplicated(keep='first')]
df_weather_temp = df_weather_temp.resample('1H').asfreq()
df_weather_temp = df_weather_temp.interpolate(method='cubic')

# You need to convert kWh to MWh by dividing it by 1000. As all other variables use MW.
df_act_loss['mwh'] = df_act_loss['mwh'].astype(float)
df_act_loss['mwh'] = df_act_loss['mwh'] / 1000.0

#downsample from 15 min to 1 hour
df_act_loss = df_act_loss.loc[~df_act_loss.index.duplicated(keep='first')]
df_act_loss = df_act_loss.resample('1H').sum()
df_act_loss = df_act_loss.interpolate()


#display
# display(df_ntc)
# display(df_act_loss)
# display(df_weather_forecast)
# display(df_weather_temp)


###Merge###
# List of DataFrames
dfs = [df_weather_forecast, df_act_loss, df_weather_temp]

# Merge all DataFrames on 'datetime'
merged_df = reduce(lambda left, right: pd.merge(left, right, on='datetime', how='inner'), dfs)

# Function to determine the season
def determine_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

# Function to determine if the date is a public holiday in Switzerland
def is_public_holiday(date):
    swiss_holidays = holidays.Switzerland(years=date.year)
    return 1 if date in swiss_holidays else 0

# Function to determine if the date is a special day in Switzerland (you can define your own special days)
def is_special_day(date):
    special_days = [pd.Timestamp(f'{date.year}-08-01'), pd.Timestamp(f'{date.year}-12-25'), pd.Timestamp(f'{date.year}-01-01')]
    return 1 if date in special_days else 0

# Add feature columns
def add_feature_columns(df):
    # Determine the season
#     df['Season'] = df.index.month.map(determine_season)
#     df = pd.get_dummies(df, columns=['Season'], prefix='Season', prefix_sep='')
#     for col in df.columns:
#         if 'Season' in col:
#             df[col] = df[col].apply(lambda x: 1 if x > 0 else 0)

    # Add month feature
    #df['Month'] = df.index.month
    #df = pd.get_dummies(df, columns=['Month'], prefix='Month')

    # Add day feature
    #df['Day'] = df.index.day  # Extract the day from the index
    #df = pd.get_dummies(df, columns=['Day'], prefix='Day')  # One-hot encode the day

    
#     # Public Holidays and Special days
#     df['Public_Holiday'] = df.index.to_series().map(is_public_holiday)
#     df['Special_Day'] = df.index.to_series().map(is_special_day)

    # Add day of the week as a one-hot encoded feature
    #df['Day_of_Week'] = df.index.dayofweek
    #df = pd.get_dummies(df, columns=['Day_of_Week'], prefix='Day')

        # Add hour of the day
    #df['Hour'] = df.index.hour
    #df = pd.get_dummies(df, columns=['Hour'], prefix='Hour')
    
    #df['dayofweek'] = df.index.strftime('%A')
    df['dayofyear'] = df.index.strftime('%j').astype(int)
    df['cosdayofyear'] = np.cos(df['dayofyear']*2*np.pi/365)
    df['sindayofyear'] = np.sin(df['dayofyear']*2*np.pi/365)
    
    return df

# Assuming merged_df is your DataFrame with datetime index
merged_df = add_feature_columns(merged_df)

#clean
merged_df.columns = merged_df.columns.str.replace('[":\[\]{}]', '', regex=True)
merged_df.columns = [col.replace(" ", "_") for col in merged_df.columns]
# display(merged_df)

df = merged_df.copy()
timeframe = 1 * 24

df['target'] = df['mwh']
df.fillna(-20, inplace=True)
df['std_mwh'] = df['mwh'].rolling(window=24).std()
# df['kurtosis'] = df['mwh'].rolling(window=24).apply(kurtosis)
# df['skewness'] = df['mwh'].rolling(window=24).apply(skew)

feature_columns = ['mwh', "std_mwh"] #drop it later

# df['kurtosis'] = df['mwh'].rolling(window=24).apply(kurtosis).shift(24)
# df['skewness'] = df['mwh'].rolling(window=24).apply(skew).shift(24)

# for col in feature_columns:
#     for i in range(1, timeframe + 1):  # Loop over 1 day's worth of hours, adjust as needed
#         lag_col_name = f'lag_{col}_{i}'  # Construct the lag column name
#         df[lag_col_name] = df[col].shift(i)  # Initially create lag columns
        
#         for idx, row in df.iterrows():
#             lag_time = idx - pd.Timedelta(hours=i)
#             # If the date portion of the lag time is the same as the current index, set it to NaN
#             if lag_time.date() == idx.date():
#                 df.at[idx, lag_col_name] = np.nan
        
for col in feature_columns:
    for i in range(1, timeframe + 1):
        lag_col_name = f'lag_{col}_{i}'
        
        # Create lag column
        df[lag_col_name] = df[col].shift(i)
        
        # Create an index of dates for each lag_time
        lag_time_index = df.index - pd.Timedelta(hours=i)
        
        # Set NaN where the date part is the same
        mask = (lag_time_index.date == df.index.date)
        df.loc[mask, lag_col_name] = np.nan
    for i in range(169, timeframe + 169):
        lag_col_name = f'lag_{col}_{i}'
        
        # Create lag column
        df[lag_col_name] = df[col].shift(i)
        
        # Create an index of dates for each lag_time
        lag_time_index = df.index - pd.Timedelta(hours=i)
        
        # Set NaN where the date part is the same
        mask = (lag_time_index.date == df.index.date)
        df.loc[mask, lag_col_name] = np.nan

# df['std_mwh_24hr'] = df['mwh'].rolling(window=24).std().shift(24)
# df['kurtosis_mwh_24hr'] = df['mwh'].rolling(window=24).apply(kurtosis).shift(24)
# df['skewness_mwh_24hr'] = df['mwh'].rolling(window=24).apply(skew).shift(24)

df = df.iloc[timeframe:]

# df = df.copy()
df.fillna(-20, inplace=True)

df = df.drop(feature_columns, axis=1)
df = df.drop(columns = ["dayofyear"])
df.to_csv('df_hua.csv')

In [33]:
df

,solar_fore_de_MW,solar_fore_it_MW,wind_fore_de_MW,wind_fore_it_MW,temperature_fore_ch,temperature_fore_fr,temperature_fore_de,temperature_fore_it,cosdayofyear,sindayofyear,...,lag_std_mwh_183,lag_std_mwh_184,lag_std_mwh_185,lag_std_mwh_186,lag_std_mwh_187,lag_std_mwh_188,lag_std_mwh_189,lag_std_mwh_190,lag_std_mwh_191,lag_std_mwh_192
datetime,,,,,,,,,,,,,,,,,,,,,
2019-01-02 00:00:00+00:00,0.0,0.0,36316.9588,2230.8105,4.014400,6.405500,4.115400,3.578600,0.999407,3.442161e-02,...,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000
2019-01-02 01:00:00+00:00,0.0,0.0,35664.6531,2138.4336,3.601709,6.023531,3.856946,3.726132,0.999407,3.442161e-02,...,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000
2019-01-02 02:00:00+00:00,0.0,0.0,35411.2512,2014.8346,2.987348,5.530419,3.568760,3.893099,0.999407,3.442161e-02,...,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000
2019-01-02 03:00:00+00:00,0.0,0.0,33715.8108,1931.9359,2.304482,5.022102,3.293981,4.129664,0.999407,3.442161e-02,...,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000
2019-01-02 04:00:00+00:00,0.0,0.0,33287.7672,1816.4149,1.686276,4.594518,3.075752,4.485993,0.999407,3.442161e-02,...,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 18:00:00+00:00,0.0,0.0,36997.7200,1108.4000,8.530000,9.960000,10.790000,9.590000,1.000000,6.432491e-16,...,21.686278,21.650642,21.320955,21.322034,21.311086,21.191685,20.579758,20.429022,20.906475,20.974073
2021-12-31 19:00:00+00:00,0.0,0.0,35666.9300,1077.9700,8.000000,9.400000,10.630000,9.110000,1.000000,6.432491e-16,...,21.687910,21.686278,21.650642,21.320955,21.322034,21.311086,21.191685,20.579758,20.429022,20.906475
2021-12-31 20:00:00+00:00,0.0,0.0,34383.8800,1048.2800,7.500000,8.880000,10.510000,8.670000,1.000000,6.432491e-16,...,21.246552,21.687910,21.686278,21.650642,21.320955,21.322034,21.311086,21.191685,20.579758,20.429022


In [38]:
df.iloc[:, 10:20]

,target,lag_mwh_1,lag_mwh_2,lag_mwh_3,lag_mwh_4,lag_mwh_5,lag_mwh_6,lag_mwh_7,lag_mwh_8,lag_mwh_9
datetime,,,,,,,,,,
2019-01-02 00:00:00+00:00,157.187522,169.801629,183.876879,196.662793,189.014544,181.348656,162.438528,170.832541,180.142571,196.673535
2019-01-02 01:00:00+00:00,156.533907,-20.000000,169.801629,183.876879,196.662793,189.014544,181.348656,162.438528,170.832541,180.142571
2019-01-02 02:00:00+00:00,163.623144,-20.000000,-20.000000,169.801629,183.876879,196.662793,189.014544,181.348656,162.438528,170.832541
2019-01-02 03:00:00+00:00,168.830854,-20.000000,-20.000000,-20.000000,169.801629,183.876879,196.662793,189.014544,181.348656,162.438528
2019-01-02 04:00:00+00:00,184.643982,-20.000000,-20.000000,-20.000000,-20.000000,169.801629,183.876879,196.662793,189.014544,181.348656
...,...,...,...,...,...,...,...,...,...,...
2021-12-31 18:00:00+00:00,167.554577,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000
2021-12-31 19:00:00+00:00,163.256620,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000
2021-12-31 20:00:00+00:00,157.771329,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000,-20.000000


# STOP

/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[lag_col_name] = df[col].shift(i)
/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[lag_col_name] = df[col].shift(i)
/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use

/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[lag_col_name] = df[col].shift(i)
/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[lag_col_name] = df[col].shift(i)
/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use

/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[lag_col_name] = df[col].shift(i)
/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[lag_col_name] = df[col].shift(i)
/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use

/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[lag_col_name] = df[col].shift(i)
/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[lag_col_name] = df[col].shift(i)
/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use

/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[lag_col_name] = df[col].shift(i)
/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[lag_col_name] = df[col].shift(i)
/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use

...

/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[lag_col_name] = df[col].shift(i)
/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[lag_col_name] = df[col].shift(i)
/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use

/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[lag_col_name] = df[col].shift(i)
/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[lag_col_name] = df[col].shift(i)
/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use

/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[lag_col_name] = df[col].shift(i)
/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[lag_col_name] = df[col].shift(i)
/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use

/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[lag_col_name] = df[col].shift(i)
/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[lag_col_name] = df[col].shift(i)
/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use

/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[lag_col_name] = df[col].shift(i)
/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[lag_col_name] = df[col].shift(i)
/tmp/ipykernel_108719/1335041470.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use

In [25]:
# from numba import cuda

# # Initialize GPU
# cuda.select_device(0)

# # Reset GPU
# cuda.close()

In [15]:

# # Sort the DataFrame by datetime
# df.sort_index(inplace=True)

# # Train-test split based on timestamp
# train_size = int(len(df) * 0.8)
# train, test = df.iloc[:train_size], df.iloc[train_size:]

# # Features and target
# X_train, y_train = train.drop('target', axis=1), train['target']
# X_test, y_test = test.drop('target', axis=1), test['target']


In [16]:
# params = {
#     'objective': 'regression',
#     'metric': 'mae',
#     'boosting_type': 'gbdt',
#     'device': 'gpu',
#     'random_state': 42,
#     'gpu_platform_id' : 0,
#     'gpu_device_id': 0
# }

# # Create LightGBM data containers
# train_data = lgb.Dataset(X_train, label=y_train)
# test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)



# # Train the model
# num_round = 50
# bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

# # Make predictions
# y_pred = bst.predict(X_test)

# # Evaluate the model
# mse = mean_squared_error(y_test, y_pred)
# rmse = np.sqrt(mse)
# mae = mean_absolute_error(y_test, y_pred)

# print(f"RMSE: {rmse}")
# print(f"MAE: {mae}")

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 44585
[LightGBM] [Info] Number of data points in the train set: 20908, number of used features: 228
[LightGBM] [Info] Using requested OpenCL platform 0 device 0
[LightGBM] [Info] Using GPU Device: NVIDIA A100 80GB PCIe, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 176 dense feature groups (3.51 MB) transferred to GPU in 0.039039 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 106.986450


KeyboardInterrupt: 

In [ ]:
# import shap

# # Initialize JavaScript for SHAP plots
# shap.initjs()

# # Create SHAP explainer
# explainer = shap.Explainer(bst)

# # Calculate SHAP values for test data
# shap_values = explainer(X_test)

# # Plot SHAP summary
# shap.summary_plot(shap_values, X_test)


In [ ]:
# # LightGBM parameters
# # Define the grid of hyperparameters to search
# param_grid = {
#     'objective': ['regression'],
#     'metric': ['mae'],
#     'boosting_type': ['gbdt'],
#     'num_leaves': [31, 127],
#     'learning_rate': [0.01, 0.1, 0.2],
#     'n_estimators': [20, 40],
#     'device': ['gpu'],
#     'gpu_platform_id': [0],
#     'gpu_device_id': [0]
# }

# # Create a LightGBM model
# lgb_estimator = lgb.LGBMRegressor()

# # Create the GridSearchCV object
# gsearch = GridSearchCV(estimator=lgb_estimator, param_grid=param_grid, 
#                        scoring='neg_mean_absolute_error', cv=3)
# # Fit the grid search to the data
# gsearch.fit(X_train, y_train)

# # Print the best parameters
# print("Best parameters found: ", gsearch.best_params_)

# # Get the best model
# best_model = gsearch.best_estimator_

# # Make predictions with the best model
# y_pred = best_model.predict(X_test)

# # Evaluate the model
# mse = mean_squared_error(y_test, y_pred)
# rmse = np.sqrt(mse)
# mae = mean_absolute_error(y_test, y_pred)

# print(f"RMSE: {rmse}")
# print(f"MAE: {mae}")
